In [1]:
import pyopenms as ms
from tools import *
import matplotlib.pyplot as plt
# %matplotlib inline

In [15]:
# mzml_blank_file="data/covert/KB20221007-1-03.mzML"
mzml_sample_file="data/covert/KB20221007-1-06.mzML"
# mzml_sample_file="data/example/FeatureFinderMetaboIdent_1_input.mzML"

exp = ms.MSExperiment()
ms.MzMLFile().load(mzml_sample_file, exp)
exp = set_ms_level(exp, 1)
# exp_base = ms.MSExperiment()
# ms.MzMLFile().load(mzml_sample_file, exp_base)
# exp_base = set_ms_level(exp_base, 1)
# print("Spectrum size: {}".format(len(exp.getSpectra())))
# for spectrum in exp_base.getSpectra():
#     exp.addSpectrum(spectrum)
# print("Spectrum size: {}".format(len(exp.getSpectra())))  

In [16]:
mass_traces = []
feature_maps = []

# Sort spectra by RT. If sort_mz=True also sort each peak in a spectrum by m/z
# exp.sortSpectra(True)
mtd = ms.MassTraceDetection()
mtd_params = mtd.getDefaults()
mtd_params.setValue(
    "mass_error_ppm", 1.0
)  # set according to your instrument mass error
mtd_params.setValue(
    "noise_threshold_int", 400.0
)  # adjust to noise level in your data
mtd.setParameters(mtd_params)
mtd.run(exp, mass_traces, 0)

mass_traces_split = []
epd = ms.ElutionPeakDetection()
epd_params = epd.getDefaults()
epd_params.setValue("width_filtering", "fixed")
epd.setParameters(epd_params)
epd.detectPeaks(mass_traces, mass_traces_split)


Progress of 'mass trace detection':
-- done [took 0.07 s (CPU), 0.08 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 0.38 s (CPU), 0.05 s (Wall)] -- 


In [27]:
# elution peak detection
mass_traces_deconvol = []
epd = ms.ElutionPeakDetection()
epd_par = epd.getDefaults()
# epd_par.setValue("width_filtering",
#                  "fixed")  # The fixed setting filters out mass traces outside the [min_fwhm: 1.0, max_fwhm: 60.0] interval
epd.setParameters(epd_par)
epd.detectPeaks(mass_traces, mass_traces_deconvol)
print(mass_traces_deconvol)

Progress of 'elution peak detection':
-- done [took 0.45 s (CPU), 0.08 s (Wall)] -- 
[<pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fb10>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873faf0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873f4b0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873f4f0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873f6d0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fa70>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873f770>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fc10>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873f510>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873f8b0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fb50>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fbd0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fcb0>, <pyopenms.pyopenms_4.Kernel_MassTrace object at 0x7fa76873fcd0>, <pyo

In [18]:
# feature detection
feature_map = ms.FeatureMap()  # output features
chrom_out = []  # output chromatograms
ffm = ms.FeatureFindingMetabo()
# ffm_par = ffm.getDefaults()
# ffm_par.setValue("remove_single_traces", "true")  # remove mass traces without satellite isotopic traces
# ffm.setParameters(ffm_par)  
ffm.run(mass_traces_deconvol, feature_map, chrom_out)
feature_map.setUniqueIds()  # Assigns a new, valid unique id per feature

Progress of 'assembling mass traces to features':
-- done [took 5.46 s (CPU), 0.81 s (Wall)] -- 


In [19]:
# hardcode
feature_map.setPrimaryMSRunPath([mzml_sample_file.encode()])  # Sets the file path to the primary MS run (usually the mzML file)
feature_xml_file = ms.FeatureXMLFile()
feature_xml_file.store("data/res.featureXML".encode(), feature_map)
                           
feature_maps.append(feature_map)

# use as reference for alignment, the file with the largest number of features (works well if you have a pooled QC for example)
# ref_index = feature_maps.index(sorted(feature_maps, key=lambda x: x.size())[-1])
# aligner = ms.MapAlignmenOtAlgorithmPoseClustering()


In [26]:
from urllib.request import urlretrieve
# gh = "https://raw.githubusercontent.com/OpenMS/pyopenms-docs/master"
# urlretrieve (gh + "/src/data/FeatureFinderCentroided_1_input.mzML", "feature_test.mzML")

from pyopenms import *

# Prepare data loading (save memory by only
# loading MS1 spectra into memory)
options = PeakFileOptions()
options.setMSLevels([1])
fh = MzMLFile()
fh.setOptions(options)


mzml_file = "data/result/KB20221007-1-05.mzML"

# Load data
input_map = MSExperiment()
fh.load(mzml_file, input_map)

input_map = make_centroid(input_map)
input_map.updateRanges()



ff = FeatureFinder()
ff.setLogType(LogType.CMD)

# Run the feature finder
name = "centroided"
features = FeatureMap()
seeds = FeatureMap()
params = FeatureFinder().getParameters(name)
ff.run(name, input_map, features, params, seeds)

features.setUniqueIds()
fh = FeatureXMLFile()
fh.store("output.featureXML", features)
print("Found", features.size(), "features")

Progress of 'Precalculating intensity scores':
Found 1 features
-- done [took 0.01 s (CPU), 0.01 s (Wall)] -- 
Progress of 'Precalculating mass trace scores':
-- done [took 0.03 s (CPU), 0.03 s (Wall)] -- 
Progress of 'Precalculating isotope distributions':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'Calculating isotope pattern scores for charge 1':
-- done [took 0.05 s (CPU), 0.05 s (Wall)] -- 
Progress of 'Finding seeds for charge 1':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Found 5 seeds for charge 1.
Progress of 'Extending seeds for charge 1':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Found 1 feature candidates for charge 1.
Progress of 'Calculating isotope pattern scores for charge 2':
-- done [took 0.14 s (CPU), 0.05 s (Wall)] -- 
Progress of 'Finding seeds for charge 2':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Found 0 seeds for charge 2.
Progress of 'Extending seeds for charge 2':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Found 0 feature ca